In [21]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

import scipy.optimize as opt

import matplotlib.pyplot as plt

%matplotlib inline

In [4]:
DATA_PATH = "./data/MC/"

In [11]:
def scan_dir(path):
    import os
    import re
    
    r = re.compile(r"\d+[_]\d+")
    
    events = list()
    
    for item in os.listdir(path):
        try:
            event_name = r.findall(item)[0]
            events.append(event_name)
        except:
            pass
    
    return list(set(events))

In [12]:
event_names = scan_dir(DATA_PATH)

In [13]:
def load_event(item, data_path = DATA_PATH):
    hits = pd.DataFrame.from_csv("%s/%s.hits.csv" % (data_path, item))[['X', 'Y', 'Z']].values
    tracks = pd.DataFrame.from_csv("%s/%s.tracks.csv" % (data_path, item))
    
    tx = tracks[[ 'x%d' % i for i in range(11) ]].values
    ty = tracks[[ 'y%d' % i for i in range(11) ]].values
    tz = tracks[[ 'z%d' % i for i in range(11) ]].values
    
    pid = tracks['name'].values
    
    return hits, (tx, ty, tz, pid)

In [14]:
def load_events(data_path = DATA_PATH):
    events = list()
    
    for item in scan_dir(data_path):
        try:
            events.append(load_event(item, data_path))
        except:
            pass

    return events

In [15]:
events = load_events()

In [16]:
len(events)

2573

In [18]:
zs = np.hstack([ hits[:, 2] for hits, _ in events ])

In [20]:
before_cut = np.max(zs[zs < 5000]) + 0.1
print before_cut

2648.4


In [36]:
def cut(events):
    from sklearn.linear_model import LinearRegression
    
    lr = LinearRegression(fit_intercept=False)
    
    for hits, (tx, ty, tz, pid) in events:
        hits_ = hits[hits[:, 2] < before_cut, :]
        
        params = np.zeros((tx.shape[0], 2))
        
        for t in xrange(tx.shape[0]):
            cut = tz[t, :] < before_cut
            
            tx_ = tx[t, :][cut]
            ty_ = ty[t, :][cut]
            tz_ = tz[t, :][cut].reshape(-1, 1)
        
            assert tz_.shape[0] > 1
        
            lr.fit(tz_, tx_)
            cx = lr.coef_
            
            print tz_, tx_
            
            assert lr.score(tz_, tx_) >= 0.98, "Low score %f" % lr.score(tz_, tx_)
        
            lr.fit(tz_, ty_)
            cy = lr.coef_
        
            assert lr.score(tz_, ty_) >= 0.98
        
            params[t, :] = cx, cy

        yield hits_, params

In [37]:
cutted = list(cut(events))

[[  -11.9815]
 [ 1389.22  ]] [  0.74088  51.255  ]
[[  -11.9815]
 [ 1389.22  ]] [  0.281882  62.9832  ]
[[  -11.9815]
 [ 1889.22  ]] [  0.520724  91.3356  ]
[[  -11.9815]
 [ 1389.22  ]] [  0.446999  78.8592  ]
[[  -11.9815]
 [ 1389.22  ]] [  0.430757  60.7484  ]
[[  -11.9815]
 [ 1389.22  ]] [  0.472913  75.8367  ]
[[  -11.9815]
 [ 1389.22  ]] [  0.357196 -23.2837  ]
[[  -11.9815]
 [ 1389.22  ]] [  0.338909 -12.8681  ]
[[  -11.9815]
 [ 1389.22  ]] [  0.41269  89.752  ]
[[   19.9478]
 [ 1417.95  ]] [   0.358979 -104.137   ]
[[  -11.9815]
 [ 1389.22  ]] [   0.359045  182.111   ]
[[  -11.9815]
 [ 1389.22  ]] [   0.530419  141.006   ]
[[  -11.9815]
 [ 1389.22  ]] [  0.315245  45.5474  ]
[[  -11.9815]
 [ 1389.22  ]] [  0.579144 -17.4054  ]
[[   19.9478]
 [ 1417.95  ]] [   0.406143 -109.05    ]
[[  -11.9815]
 [ 1389.22  ]] [  0.500624 -36.094   ]
[[  -11.9815]
 [ 1389.22  ]] [  0.456859  43.2812  ]
[[  -11.9815]
 [ 1889.22  ]] [   0.186224  169.645   ]
[[   19.9478]
 [ 1417.95  ]] [  0.239901

AssertionError: Low score 0.497478